# Convert YoloV5 to ONNX on SageMaker Notebook Instance

The official document is [here](https://github.com/ultralytics/yolov5/issues/251). Here is demostration how to use SageMaker jupyter notebook to export YoloV5 model to ONNX.

In [ ]:
!pip install tensorboard onnx torch torchvision --upgrade

In [ ]:
# clean up if loaded yolov5
# !rm -rf yolov5
# !rm *.onnx *.pt

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [1]:
# Add yolov5 system path
import sys, os, onnx, torch

In [2]:
# must 1.5+
print('\nStarting TorchScript export with torch %s...' % torch.__version__)


Starting TorchScript export with torch 1.5.1...


In [3]:
fp_yolov5 = os.path.abspath(os.getcwd()) + "/yolov5"
sys.path.append(fp_yolov5)
sys.path

['',
 '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python36.zip',
 '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6',
 '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/lib-dynload',
 '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages',
 '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/IPython/extensions',
 '/home/ec2-user/.ipython',
 '/home/ec2-user/SageMaker/myAWSStudyBlog/sagemaker-greengrass/0-yolov5-onnx/yolov5']

### Change working directory to yolov5

In [4]:
%cd ./yolov5

/home/ec2-user/SageMaker/myAWSStudyBlog/sagemaker-greengrass/0-yolov5-onnx/yolov5


# Download pre-trained models

In [5]:
!python -c "from utils.google_utils import *; attempt_download('weights/yolov5s.pt');"

In [6]:
!ls weights

download_weights.sh  yolov5s.pt


# Export a Trained YoloV5 Model

In [7]:
img = torch.zeros((1, 3, 640, 640))
fp_model = './weights/yolov5s.pt'

In [8]:
model = torch.load(fp_model, map_location=torch.device('cpu'))['model'].float()
model.eval()
model.model[-1].export = True # set Detect() layer export = True, which will not include detect layer
y = model(img)  # dry run, to inference mode

## torchscript

In [9]:
fp_torchscript = './yolov5s.torchscript.pt'  # filename
ts = torch.jit.trace(model, img)
ts.save(fp_torchscript)

## onnx model

In [10]:
fp_onnx = './yolov5s.onnx'  # filename
# set Detect() layer export=False, ref [issue343](https://github.com/ultralytics/yolov5/issues/343#issuecomment-658021043) 
model.fuse()  # only for ONNX
torch.onnx.export(model, img, fp_onnx, verbose=False, opset_version=12, input_names=['images'],
                          output_names=['classes', 'boxes'] if y is None else ['output'])

Fusing layers... Model Summary: 140 layers, 7.45958e+06 parameters, 7.45958e+06 gradients


## onnx model with detect layer

In [11]:
fp_onnx_detect = './yolov5s_detect.onnx'  # filename

model = torch.load(fp_model, map_location=torch.device('cpu'))['model'].float()
model.eval()
# ref [issue343](https://github.com/ultralytics/yolov5/issues/343#issuecomment-658021043)
model.model[-1].export = False
y = model(img)  # dry run to inference mode
model.fuse()  # only for ONNX
torch.onnx.export(model, img, fp_onnx_detect, verbose=False, opset_version=12, input_names=['images'],
                          output_names=['classes', 'boxes'] if y is None else ['output'])

Fusing layers... Model Summary: 140 layers, 7.45958e+06 parameters, 7.45958e+06 gradients


/home/ec2-user/SageMaker/myAWSStudyBlog/sagemaker-greengrass/0-yolov5-onnx/yolov5/models/yolo.py:30: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.grid[i].shape[2:4] != x[i].shape[2:4]:


The warning of the output is because of inference: https://github.com/ultralytics/yolov5/blob/master/models/yolo.py#L29-L36

In [12]:
onnx_model = onnx.load(fp_onnx_detect)  # load onnx model
onnx.checker.check_model(onnx_model)  # check onnx model
# print(onnx.helper.printable_graph(onnx_model.graph))  # print a human readable model
print('ONNX export success, saved as %s' % fp_onnx_detect)

ONNX export success, saved as ./yolov5s_detect.onnx


### Move tourch script and ONNX model out

In [13]:
!cp {fp_torchscript} ../yolov5s.torchscript.pt
!cp {fp_onnx} ../yolov5s.onnx
!cp {fp_onnx_detect} ../yolov5s_detect.onnx

In [ ]:
# use https://lutzroeder.github.io/netron/ to check the model